In [1]:
from multiprocessing import Pool
import math, random, sys
sys.path.insert(0, '/home/marcase/hgraph2graph/')
import pickle
import argparse
from functools import partial
import torch
import numpy
from hgraph import MolGraph, common_atom_vocab, PairVocab
import rdkit

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
def to_numpy(tensors):
    convert = lambda x : x.numpy() if type(x) is torch.Tensor else x
    a,b,c = tensors
    b = [convert(x) for x in b[0]], [convert(x) for x in b[1]]
    return a, b, c

def tensorize(mol_batch, vocab,max_span_tree=True):
    x = MolGraph.tensorize(mol_batch, vocab, common_atom_vocab,max_span_tree)
    return to_numpy(x)

In [25]:
lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

vocab = '/home/marcase/hgraph2graph/data/cyclic_peptides/cyclic_vocab_new.txt'
vocab = [x.strip("\r\n ").split() for x in open(vocab)]
vocab = PairVocab(vocab)
vocab.vocab[21][0]
vocab.hvocab

['CC',
 'CC(C)C',
 'CC(C)O',
 'CCC(=O)O',
 'CCC(C)C',
 'CCC(N)=O',
 'CCC1=CC=C(O)C=C1',
 'CCC1=CC=CC=C1',
 'CCC1=CN=CN1',
 'CCC1=CNC2=C1C=CC=C2',
 'CCCC(=O)O',
 'CCCC(N)=O',
 'CCCCCN',
 'CCCCNC(=N)N',
 'CCCSC',
 'CCO',
 'CCSCC(=O)CSCC',
 'CN1CCCC1C(=O)O',
 'CN1CCCC1C=O',
 'CNCC(=O)O',
 'CNCC=O',
 'NCC=O',
 'O=CC1CCCN1']

In [26]:
class Args:
    train = '/home/marcase/hgraph2graph/predict/X_train.txt'
    train_labels = '/home/marcase/hgraph2graph/predict/y_train.txt'
    test = '/home/marcase/hgraph2graph/predict/X_test.txt'
    test_labels = '/home/marcase/hgraph2graph/predict/y_test.txt'
    vocab = vocab
    seed = 7
    max_span_tree = True
    ncpu = 16
    mode = 'single'
    batch_size = 32
    
args=Args()

In [27]:
pool = Pool(args.ncpu) 
random.seed(1)

In [28]:
#dataset contains single molecules
with open(args.train) as f:
    data = [line.strip("\r\n ").split()[0] for line in f]
with open(args.train_labels) as f:
    data_labels = [int(line.strip("\r\n ").split()[0]) for line in f]

In [29]:
random.shuffle(data)
random.shuffle(data_labels)

In [30]:
batches = [data[i : i + args.batch_size] for i in range(0, len(data), args.batch_size)]
batches_labels = [data_labels[i : i + args.batch_size] for i in range(0, len(data_labels), args.batch_size)]

In [31]:
for i in range(len(batches)):
    print('Batch ' + str(i) + ' data: ' + str(len(batches[i])) + ' labels: ' + str(len(batches_labels[i])))

Batch 0 data: 32 labels: 32
Batch 1 data: 32 labels: 32
Batch 2 data: 32 labels: 32
Batch 3 data: 32 labels: 32
Batch 4 data: 32 labels: 32
Batch 5 data: 32 labels: 32
Batch 6 data: 32 labels: 32
Batch 7 data: 32 labels: 32
Batch 8 data: 32 labels: 32
Batch 9 data: 32 labels: 32
Batch 10 data: 32 labels: 32
Batch 11 data: 32 labels: 32
Batch 12 data: 32 labels: 32
Batch 13 data: 32 labels: 32
Batch 14 data: 32 labels: 32
Batch 15 data: 32 labels: 32
Batch 16 data: 32 labels: 32
Batch 17 data: 32 labels: 32
Batch 18 data: 32 labels: 32
Batch 19 data: 32 labels: 32
Batch 20 data: 32 labels: 32
Batch 21 data: 32 labels: 32
Batch 22 data: 32 labels: 32
Batch 23 data: 32 labels: 32
Batch 24 data: 32 labels: 32
Batch 25 data: 32 labels: 32
Batch 26 data: 32 labels: 32
Batch 27 data: 32 labels: 32
Batch 28 data: 32 labels: 32
Batch 29 data: 32 labels: 32
Batch 30 data: 32 labels: 32
Batch 31 data: 32 labels: 32
Batch 32 data: 32 labels: 32
Batch 33 data: 32 labels: 32
Batch 34 data: 32 labels

In [32]:
func = partial(tensorize, vocab = args.vocab,max_span_tree=args.max_span_tree)
all_data = pool.map(func, batches)

KeyboardInterrupt: 

In [ ]:
print(len(all_data))
num_splits = len(all_data) // 1000
if num_splits < 1000 and num_splits >= 100:
    num_splits = len(all_data) // 100
elif num_splits < 100:
    num_splits = len(all_data) // 10
print(num_splits)
le = (len(all_data) + num_splits - 1) // num_splits

for split_id in range(num_splits):
    st = split_id * le
    sub_data = all_data[st : st + le]

    with open('tensors-%d.pkl' % split_id, 'wb') as f:
        pickle.dump(sub_data, f, pickle.HIGHEST_PROTOCOL)